<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/C%C3%B3digo_en_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
library(ggplot2)
#install.packages('dplyr')
library(dplyr)
#install.packages('MASS')
library(MASS)
#install.packages('randomForest')
library(randomForest)
#install.packages('caret')
library("caret")
library("googledrive")


In [19]:
id <- "1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE" # google file ID
df <- read.csv(sprintf("https://docs.google.com/uc?id=%s&export=download", id))
str(df)

'data.frame':	40000 obs. of  25 variables:
 $ X                    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ vigente              : num  4282 9793 531420 0 112394 ...
 $ vigenteAvg3M         : num  4679 9451 512057 0 110856 ...
 $ vencido              : num  0 83970 0 2267 0 ...
 $ vencidoAvg3M         : num  0 91127 0 2633 0 ...
 $ avgcontingenciaU1M   : num  4923 20809 36813 11534 41131 ...
 $ avgcontingenciaU3M   : num  3919 19127 46144 10488 42820 ...
 $ sumcontingenciaU1M   : num  19692 41618 36813 23068 82263 ...
 $ sumcontingenciaU3M   : num  18440 49280 36789 24026 77614 ...
 $ maxcontingenciaU1M   : num  6653 30881 36813 19045 43945 ...
 $ maxcontingenciaU3M   : num  7996 32794 46144 17864 42820 ...
 $ numcontingenciaU1M   : int  4 2 1 2 2 1 0 2 4 0 ...
 $ garantiacomputable   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ peorcalificacionBCU  : chr  "1C" "3" "1C" "5" ...
 $ numeroinstituciones  : int  4 2 3 4 4 3 2 6 4 1 ...
 $ numerobancosprivados : int  1 0 2 4 4 0 0 0 0 0 ...
 $ numerobancosp

In [ ]:
#Convertir variables cualitativas

df$Tipo_Contrato2 = ifelse(data$Tipo_Contrato == 'Prepago', 1, 
ifelse(data$Tipo_Contrato == 'Basico',2, ifelse(data$Tipo_Contrato == 'Superior',3,4)))
unique(data$Tipo_Contrato2)


data_num <- data[,-c(3,5,10)]
data_num

In [ ]:
# Transformación de las variables categóricas en factor

data_num <- transform(data_num, Class = as.factor(Class), 
Debito_autom =as.factor(Debito_autom), Recibio_oferta = as.factor(Recibio_oferta), 
SEXO2 = as.factor(SEXO2), Anos_como_cliente2 = as.factor(Anos_como_cliente2), 
Numero_quejas = as.factor(Numero_quejas), Meses_adeudo = as.factor(Meses_adeudo))

str(data_num)

In [ ]:
# Partición del dataset entre train (80%) y test (20%)

set.seed(77)
data_index = createDataPartition(data_num$Class, p=0.80, list = FALSE)
data_train <- data_num[data_index, ]
data_test <- data_num[-data_index, ]

In [ ]:
# Entrenamiento del modelo

rf = randomForest(Class ~ ., data = data_train, mtry = 3, importance = TRUE, ntree = 500)

rf_pred = predict(rf, newdata = data_test[,-1], type="prob")

rf_tabla = table(ifelse(rf_pred[,2]>0.5,1,0), data_test[,1])
rf_tabla
rf_er = 1- sum(diag(rf_tabla))/sum(rf_tabla)
rf_er
rf_er = rf_er*100

In [6]:
# Optimización del modelo Random Forest

set.seed(77)

k_folds = 5
folds = caret::createFolds(c(1:nrow(data_num)),k=k_folds)
m = seq(1,10, by=2)
narbol = seq(100, 1000, by = 100)
cutoff = c(seq(0.40, 0.70, by = 0.05))
resultadosrf = data.frame()

set.seed(77)

for (k in 1:k_folds){
  for (j in m){
    for (i in narbol){
      for (f in cutoff){
      train = data_num[-folds[[k]],]
      test = data_num[folds[[k]],]
    rf_opt = randomForest(Class ~ ., data = train, mtry = j, importance = TRUE, ntree = i)
    rf_pred_opt = predict(rf_opt, newdata = test[,-1], type="prob")
    rf_tabla_opt = table(ifelse(rf_pred_opt[,2]>f,1,0), test[,1])
    rf_er_opt = 1- sum(diag(rf_tabla_opt))/sum(rf_tabla_opt)
    result = c("k_folds"=k, "m"=j, "narbol"=i, "cutoff"=f, "error"= rf_er_opt)
    resultadosrf = rbind(resultadosrf, result)
  }
  }
}}


resultadosrf <- setNames(resultadosrf,c("k_folds","m","narbol","cutoff","error"))
resultadosrf_cv <- aggregate(resultadosrf[,5],list(resultadosrf$m,resultadosrf$narbol,resultadosrf$cutoff),mean)
resultadosrf_cv <- setNames(resultadosrf_cv,c("m","narbol","cutoff","error"))

#View(resultadosrf_cv)

resultadosrf_cv_ord <- resultadosrf_cv[order(resultadosrf_cv$error),]
head(resultadosrf_cv_ord)

ERROR: ignored

In [ ]:
importance(rf)
varImpPlot(rf)

In [ ]:
# Predicción con el modelo optimizado

rf_pred_def = predict(rf_final, newdata = data_testchurn_num, type="prob")
#View(rf_pred_def)

df_def = data.frame(ifelse(rf_pred_def[,2]>corte,1,0))
df_def <- setNames(df_def, "prediccion")
#View(df_def)

write.csv(x = df_def, file = "df_def.csv", row.names = TRUE) 